In [39]:
import pandas as pd
df = pd.read_csv('prompts2.csv', sep=';')
X, y = df['Prompt'].values, df['Flag'].values
print(f'Załadowano {len(X)} próbek.')

Załadowano 60 próbek.


In [16]:
from transformers import pipeline

In [36]:
pipe_mod = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
res = pipe_mod(X[0])
print(f"Zdanie {X[0]} ma sentyment: {res[0]['label']}, a oczekiwana odpowiedź to: {y[0]}")

Device set to use cpu


Zdanie Rosja ma większą powierzchnię niż planeta Pluton ma sentyment: NEGATIVE, a oczekiwana odpowiedź to: 1
